Parameters that are not tuned during training. The hyperparameter that should be tuned are within optunas objective() function.

In [1]:
BATCH_SIZE = 30
EPOCHS = 50
NUM_WORKERS = 2 # number of CPUs used
NTH_FRAME = 2
SLIDING_STEP = 1
SEQ_LEN = 1

DIR = "data/30x25/"

In [2]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback

import torch
import pytorch_lightning as pl

from models.decoder.architectures import LightningNet
from data.gdm_dataset import GasDataModule

def objective(trial):
    num_layers = 5
    learning_rate = trial.suggest_float("lr", 1e-5, 1e-2)
    inner_dims = [trial.suggest_int("inner_dims_{}".format(i), 3, 256, log=True) for i in range(num_layers)]

    model = LightningNet(inner_dims, SEQ_LEN, learning_rate)
    datamodule = GasDataModule(data_dir=DIR, seq_len=SEQ_LEN, nth_frame=NTH_FRAME, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
    
    tb_logger = pl.loggers.TensorBoardLogger(
        save_dir="logs", 
        name="decoder",
        #version=
    )
    trainer = pl.Trainer(
        logger = tb_logger,
        min_epochs = int(EPOCHS/2),
        max_epochs = EPOCHS,
        gpus=1 if torch.cuda.is_available() else None,
        #callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_loss")],
        enable_progress_bar=False,
        enable_model_summary=False,
        fast_dev_run=True # set to False, if model should be trained
    )
    
    trainer.fit(model, datamodule=datamodule)
    
    return trainer.callback_metrics["val_loss"].item()


optuna.logging.set_verbosity(optuna.logging.ERROR)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1, show_progress_bar=True)

/home/nicolas/miniconda3/envs/torch/lib/python3.10/site-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/1 [00:00<?, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
/home/nicolas/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/nicolas/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.